In [ ]:
def new_bit_board(rows=6, cols=7, sequence=4, gravity=True, turn=0):
    rows_b = (rows & 15) << 0 # 4 bits
    cols_b = (cols & 15) << 4 # 4 bits
    sequence_b = (sequence & 15) << 8 # 4 bits
    gravity_b = (gravity & 1) << 12 # 1 bit
    turn_b = (turn & 1) << 13 # 1 bit
    return rows_b | cols_b | sequence_b | gravity_b | turn_b

def get_sub_bits(bits, n_bits, start):
    return (bits & (((1 << n_bits) - 1) << start)) >> start

def get_rows(bit_board): return get_sub_bits(bit_board, 4, 0)
def get_cols(bit_board): return get_sub_bits(bit_board, 4, 4)
def get_win_con(bit_board): return get_sub_bits(bit_board, 4, 8)
def get_gravity(bit_board): return get_sub_bits(bit_board, 1, 12)
def get_turn(bit_board): return get_sub_bits(bit_board, 1, 13)
def get_board(bit_board, player=0):
    rows_cols = get_rows(bit_board) * get_cols(bit_board)
    return get_sub_bits(bit_board, rows_cols, 14 + rows_cols * player)


In [ ]:
RULES = {
    "tic-tac-toe": {
        "rows": 3,
        "cols": 3,
        "sequence": 3,
        "gravity": False,
    },
    "connect-4": {
        "rows": 6,
        "cols": 7,
        "sequence": 4,
        "gravity": True,
    }
}

bit_board = new_bit_board(**RULES["tic-tac-toe"])
bin(bit_board)

In [ ]:
def row_bits(bit_board, row_i):
    cols = get_cols(bit_board)
    row_mask = ((1 << cols) - 1) << (row_i * cols)
    return [row_mask & board for board in [get_board(bit_board, player) for player in [0, 1]]]

# TODO: left-off here
#def col_bits(rules, col_i):
#    rows, cols = get_rows(rules), get_cols(rules)
#    mask = 0
#    for i in range(rows):
#        mask |= 1 << (cols * i)
#    return mask << col_i



In [ ]:
def flip_bit_board(board, rules, direction="horizontal"):
    flipped_board = new_bit_board(rules)
    if direction == "horizontal":
        cols = get_cols(rules)
        for i in range(cols):
            mask = board & column_bit_mask(rules, i)
            flipped_board |= (mask >> i) << (cols - 1 - i)
    else:
        rows, cols = get_rows(rules), get_cols(rules)
        for i in range(rows):
            mask = board & row_bit_mask(rules, i)
            flipped_board |= (mask >> i * cols) << ((rows - 1 - i) * cols)
    return flipped_board

In [ ]:
def diagonal_bit_board(board, rules):
    rows, cols = get_rows(rules), get_cols(rules)
    # fix

board = new_bit_board(new_bit_rules())
board |= 1 | 8 | 64
bin(diagonal_bit_board(board))

In [ ]:
def get_row_mask(row_index, num_cols=7):
    return ((1 << num_cols) - 1) << (num_cols * row_index)

def get_column_mask(col_index, num_cols=7):
    mask = 0
    for row in range(6):
        mask |= 1 << (7 * row)
    return mask << (col_index)

def check_row(row, sequence):
    mask = (1 << sequence) - 1
    while row:
        if row < mask: return False
        if (row & mask) == mask: return True
        row >>= 1
    return False

col0 = get_column_mask(0)
col1 = get_column_mask(1)
col2 = get_column_mask(2)
col3 = get_column_mask(3)
col4 = get_column_mask(4)
col5 = get_column_mask(5)
col6 = get_column_mask(6)

row = get_row_mask(5)
bin(row)